# Обучения языковых моделей

В этом ноутбуке будет произведено обучение языковых моделей для модели итеративного исправления. Требуется обучить две модели:

1. Слева-направо
2. Справа-налево

В качестве обучающего корпуса будет взят корпус Тайга. Там есть части из соц.сетей, новостных сайтов, субтитров,что должно быть достаточно близким к изучаемому доменом.

В качестве модели было решено взять KenLM в виду скорости и неплохого качества.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import sys
import os
import re
from string import punctuation
from collections import Counter
sys.path.append('..')

import dotenv
import numpy as np
import pandas as pd

import nltk
from sacremoses import MosesTokenizer, MosesDetokenizer

from IPython.display import display
from tqdm.notebook import tqdm

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/mrgeekman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
PROJECT_PATH = os.path.join(os.path.abspath(''), os.pardir)
CONFIGS_PATH = os.path.join(PROJECT_PATH, 'src', 'configs')
os.environ['DP_PROJECT_PATH'] = PROJECT_PATH

## Данные

В качестве данных решено было задействовать все данные корпуса "Тайга". Все файлы для скачивания доступны по [ссылке](https://tatianashavrina.github.io/taiga_site/downloads).

## Подготовка

В первую очередь требуется предобработать все тексты, что у нас имеются. Согласно задаче, нас не интересует регистр слов и пунктуация, поэтому избавимся от нее. В качестве результата должны получиться два текстовых файла (прямой и обратный), где каждое предложение расположено на отдельной строчке -- именно в таком виде следует подавать данные для обучающей программы.

In [5]:
!mkdir ../data/processed/kenlm -p

In [6]:
DATA_PATH = os.path.join(PROJECT_PATH, 'data')
TAIGA_PATH = os.path.join(DATA_PATH, 'external', 'taiga')
RESULT_PATH = os.path.join(DATA_PATH, 'processed', 'kenlm')
MODEL_PATH = os.path.join(PROJECT_PATH, 'models')

In [7]:
tokenizer = MosesTokenizer(lang='ru')
detokenizer = MosesDetokenizer(lang='ru')

Напишем функцию, которая будет брать строчки с предложениями из файлов `texts_tagged`, убирать из них пунктуацию и записывать в файл.

In [8]:
def write_sentences(lines, out_file_left_right, out_file_right_left):
    """Чтение, обработка, запись предложений по строчкам в texts_tagged."""
    with open(out_file_left_right, 'a') as ouf_left_right:
        with open(out_file_right_left, 'a') as ouf_right_left:
            for line in lines[1:]:
                if line.startswith('# text = '):
                    sentence = line[len('# text = '):].strip().lower()
                    tokenized_sentence = tokenizer.tokenize(
                        sentence, escape=False
                    )
                    cleaned_tokenized_sentence = [
                        x for x in tokenized_sentence 
                        if not re.fullmatch('[' + punctuation + ']+', x)
                    ]
                    ouf_left_right.write(
                        detokenizer.detokenize(
                            cleaned_tokenized_sentence
                        )
                        + '\n'
                    )
                    ouf_right_left.write(
                        detokenizer.detokenize(
                            cleaned_tokenized_sentence[::-1]
                        )
                        + '\n'
                    )

Теперь напишем функцию, которая будет читать данные непосредственно из директории.

In [9]:
def write_sentences_from_dir(dir_path, out_file_left_right, out_file_right_left):
    """Чтение, обработка, запись преложений из директории."""
    for filename in tqdm(sorted(os.listdir(dir_path))):
        file_path = os.path.join(dir_path, filename)
        with open(file_path, 'r') as inf:
            lines = inf.readlines()
            write_sentences(lines, left_right_path, right_left_path)

### Arzamas

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'Arzamas', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'arzamas_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'arzamas_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

### NPlus1

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'NPlus1', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'nplus1_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'nplus1_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

### Новости

#### Fontanka

Сначала добавим только Фонтанку, потому что там есть деление по годам.

In [ ]:
FONTANKA_PATH = os.path.join(TAIGA_PATH, 'Fontanka', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'news_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'news_right_left.txt')

In [ ]:
for year in tqdm(sorted(os.listdir(FONTANKA_PATH))):
    year_path = os.path.join(FONTANKA_PATH, year)
    write_sentences_from_dir(year_path, left_right_path, right_left_path)

#### Interfax

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'Interfax', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'news_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'news_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

#### KP

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'KP', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'news_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'news_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

#### Lenta

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'Lenta', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'news_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'news_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

### Соцсети

Теперь обработаем тексты из соцсетей. Насчет включения этого раздела я до сих пор сомневаюсь. Тут весьма специфичный вокабуляр и достаточно много опечаток самих по себе.

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'social', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'social_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'social_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

### Субтитры

Обработаем тексты из субтитров.

Особенность обработки в том, что в данных помимо текста есть таймкоды. Также одно и то же предложение в общем случае разбито на несколько таймкодов. Поэтому придется научиться фильтровать таймкоды при помощи регулярных выражений.

Загружать таблицу не понадобится, так как в `tagged_texts` уже лежат только субтитры на русском языке.

In [ ]:
SUBTITLES_PATH = os.path.join(TAIGA_PATH, 'Subtitles', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'subtitles_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'subtitles_right_left.txt')

In [ ]:
for title in tqdm(sorted(os.listdir(SUBTITLES_PATH))):
    title_path = os.path.join(SUBTITLES_PATH, title)
    for filename in sorted(os.listdir(title_path)):
            file_path = os.path.join(title_path, filename)
            with open(file_path, 'r') as inf:
                lines = inf.readlines()
                edited_lines = [
                    re.sub(
                        '\d+ \d\d:\d\d:\d\d,\d\d\d \d\d:\d\d:\d\d,\d\d\d', 
                        '', 
                        x
                    )
                    for x in lines
                ]
                write_sentences(edited_lines, left_right_path, right_left_path)

### Magazines

In [ ]:
cur_path = os.path.join(TAIGA_PATH, 'Magazines', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'magazines_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'magazines_right_left.txt')

In [ ]:
write_sentences_from_dir(cur_path, left_right_path, right_left_path)

### Stihi

Этот раздел решено было пропустить.

In [ ]:
STIHI_PATH = os.path.join(TAIGA_PATH, 'stihi_ru', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'stihi_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'stihi_right_left.txt')

In [ ]:
for year in tqdm(sorted(os.listdir(STIHI_PATH))):
    year_path = os.path.join(STIHI_PATH, year)
    for month in sorted(os.listdir(year_path)):
        month_path = os.path.join(year_path, month)
        write_sentences_from_dir(month_path, left_right_path, right_left_path)

### Proza

Из этого раздело решено было взять только последние 3 года.

In [ ]:
PROZA_PATH = os.path.join(TAIGA_PATH, 'proza_ru', 'tagged_texts')
left_right_path = os.path.join(RESULT_PATH, 'proza_left_right.txt')
right_left_path = os.path.join(RESULT_PATH, 'proza_right_left.txt')

In [ ]:
for year in tqdm(sorted(os.listdir(PROZA_PATH))[-3:]):
    year_path = os.path.join(PROZA_PATH, year)
    for month in tqdm(sorted(os.listdir(year_path))):
        month_path = os.path.join(year_path, month)
        write_sentences_from_dir(month_path, left_right_path, right_left_path)

### Сборка обучающего датасета

Из всего выше было решено взять:
* Arzamas
* NPlus1
* Новости
* Соцсети
* Субтитры
* Magazines
* Proza

Последний пункт был взят не полностью, а просто чтобы получить в итоге 50/100 млн предложений. Так было решено сделать, чтобы не слишком сильно раздувать размер языковой модели. В противном случае, она просто не поместится в оперативную память даже при использовании бора и квантизации.

Теперь можем сконкатенируем полученные файлы для обучения языковых моделей. Для этого проще всего использовать команду `cat`.

Датасет на 50 млн. предложений вести примерно 6.4 ГБ, а датасет на 100 млн. примерно 12.5.

## Обучение

Теперь выполним обучение. Для этого вспользуемя [документацией](https://kheafield.com/code/kenlm/estimation/) и [инструкцией](https://github.com/kmario23/KenLM-training).

На этом этапе подразумевается, что библиотека уже склонирована в src/kenlm и собрана.

Код ниже -- это просто демонстрация как может выглядеть обучение для одной конкретной языковой модели.

Я обучал несколько вариаций, где варьировал
1. Количество данных для обучения.
2. Наличие прунинга.
3. Наличие фильтрации по словарю.
4. Порядок языковой модели (3 или 4).
5. Использование данных из соц.сетей при обучении.

In [ ]:
!../src/kenlm/build/bin/lmplz -o 3 -S 80% -T /tmp < ../data/processed/kenlm/left_right.txt > ../models/kenlm/left_right.arpa

### Фильтрация

Этот шаг существует для того, чтобы убрать из модели те слова/n-граммы, которых нет в целевом датасете, что позволяет уменьшить вес модели и время загрузки: [документация](https://kheafield.com/code/kenlm/filter/).

В нашем случае можно попробовать выполнить фильтрацию по используемому словарю и исключить те n-граммы, которые включают неизвестные слова.

Посмотрим на размеры бинаризованных моделей без фильтрации.

In [ ]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/left_right_3_50.arpa

Выполним фильтрацию.

In [ ]:
!cat ../data/external/russian_words/russian_words_vocab.dict | ../src/kenlm/build/bin/filter single model:../models/kenlm/left_right_3_50.arpa ../models/kenlm/left_right_3_50_filtered.arpa

Посмотрим на требуемый объем памяти для различных способов бинаризации.

In [ ]:
!../src/kenlm/build/bin/build_binary ../models/kenlm/left_right_3_50_filtered.arpa

### Бинаризация

Возьмем бинаризацию с построением бора, потому что она требует меньше памяти.

In [ ]:
!../src/kenlm/build/bin/build_binary trie ../models/kenlm/left_right_3_50.arpa ../models/kenlm/left_right_3_50.arpa.binary

## Тест

А теперь загрузим модель и попробуем применить ее к какому-либо предложению.

In [10]:
import kenlm

model_left_right = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'left_right_3_50.arpa.binary')
)
model_right_left = kenlm.LanguageModel(
    os.path.join(MODEL_PATH, 'kenlm', 'right_left_3_50.arpa.binary')
)

In [11]:
example = 'журналисты всегда все нагло беспардонно переврут'
example_reversed = ' '.join(example.split(' ')[::-1])

In [12]:
%%timeit
model_left_right.score(example)

1.57 µs ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
model_left_right.score(example)

-26.72813606262207

In [14]:
model_right_left.score(example_reversed)

-26.647722244262695